In [2]:
! pip install mesa --quiet
! pip install flask --quiet


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\samir\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\samir\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Librerías de simulación multiagentes
from mesa import Agent, Model  # Define los agentes y el modelo base de la simulación
from mesa.space import MultiGrid, SingleGrid  # Permite colocar los agentes en una cuadrícula (multi o individual)
from mesa.time import SimultaneousActivation  # Activa a todos los agentes simultáneamente en cada paso de la simulación
from mesa.datacollection import DataCollector  # Recolecta y organiza datos de la simulación para análisis

# Librerías matemáticas y generación de aleatoriedad
import itertools  # Proporciona herramientas para crear combinaciones y permutaciones
import random  # Permite generar números y secuencias aleatorias, útil para la variabilidad en simulaciones
import math  # Contiene funciones matemáticas básicas, como operaciones trigonométricas y logarítmicas

# Librerías de visualización y gráficos
import matplotlib.pyplot as plt  # Creación y personalización de gráficos
import matplotlib.patches as patches  # Añade formas como rectángulos o círculos a gráficos de matplotlib
from matplotlib.colors import ListedColormap  # Define mapas de color personalizados
import matplotlib.animation as animation  # Crea animaciones, útil para representar dinámicamente la simulación
plt.rcParams["animation.html"] = "jshtml"  # Configura la visualización de animaciones en formato HTML
import matplotlib  # Configuración adicional de matplotlib
matplotlib.rcParams["animation.embed_limit"] = 2**128  # Ajusta el límite de tamaño para incrustar animaciones

# Librerías de manipulación y análisis de datos
import numpy as np  # Proporciona funciones para manejo de matrices y álgebra lineal, común en análisis de datos
import pandas as pd  # Ofrece estructuras de datos como DataFrames, útiles para manipular grandes cantidades de datos
import time  # Proporciona funciones para trabajar con fechas y horas

# Librerías de comunicación en red y servidor HTTP
from http.server import BaseHTTPRequestHandler, HTTPServer  # Implementa un servidor HTTP básico para manejar solicitudes
import logging  # Registro de eventos y errores del servidor, útil para el diagnóstico
import json  # Proporciona funciones para manejar datos en formato JSON, útil para comunicación entre aplicaciones

import heapq

In [ ]:
class LuigiAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.action_points = 4
        self.saved_points = 0  # Puntos ahorrados de turnos anteriores
        self.carrying_portrait = None  # Para saber si está cargando un retrato verdadero

    def calculate_action_points(self):
        """Calcula los puntos de acción disponibles al inicio del turno."""
        self.action_points += self.saved_points
        if self.action_points > 8:  # No puede exceder 8 puntos
            self.action_points = 8
        self.saved_points = 0  # Los puntos ahorrados se usan al inicio del turno

    def move(self, direction):
        """Mueve al agente en una dirección, si tiene puntos suficientes."""
        if self.action_points >= 1:
            # Implementar lógica de movimiento aquí, según el modelo
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para moverse.")

    def open_close_door(self, door):
        """Abre o cierra una puerta."""
        if self.action_points >= 1:
            # Implementar lógica de abrir/cerrar puerta aquí
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para abrir/cerrar la puerta.")
    
    def break_wall(self, wall):
        """Rompe una pared."""
        if self.action_points >= 2:
            # Implementar lógica de romper pared aquí
            self.action_points -= 2
        else:
            print("No hay suficientes puntos de acción para romper la pared.")

    def knock_out_ghost(self, ghost):
        """Derriba un fantasma."""
        if self.action_points >= 1:
            # Implementar lógica para derribar al fantasma
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para noquear al fantasma.")

    def kill_ghost(self, ghost):
        """Mata un fantasma."""
        if self.action_points >= 2:
            # Implementar lógica para matar al fantasma
            self.action_points -= 2
        else:
            print("No hay suficientes puntos de acción para matar al fantasma.")

    def examine_portrait(self, portrait):
        """Examina un retrato."""
        if self.action_points >= 1:
            # Implementar lógica para examinar retrato
            self.action_points -= 1
        else:
            print("No hay suficientes puntos de acción para examinar el retrato.")

    def move_with_portrait(self, direction):
        """Se mueve cargando un retrato."""
        if self.carrying_portrait:
            if self.action_points >= 2:
                # Implementar lógica para moverse cargando el retrato
                self.action_points -= 2
            else:
                print("No hay suficientes puntos de acción para moverse cargando un retrato.")
        else:
            print("No está cargando ningún retrato.")

    def decide_to_save_points(self):
        """Decide si ahorrar puntos de acción basándose en la estrategia."""
        # Prioridad 1: Si está cargando un retrato, mover hacia la salida es crítico
        if self.carrying_portrait:
            return False  # No ahorra puntos, necesita moverse hacia la salida.

        # Prioridad 2: Hay fuego cerca de víctimas o áreas importantes
        fire_nearby = self.model.check_fire_nearby(self.pos)
        if fire_nearby:
            return False  # Actuar en fuego cercano antes de ahorrar puntos.

        # Prioridad 3: Si no hay acciones urgentes, ahorrar puntos
        if self.action_points <= 2:
            # Si hay pocos puntos restantes, es más estratégico ahorrar
            return True

        # Prioridad 4: Condiciones generales para ahorrar
        return True  # Por defecto, ahorrar puntos si no hay prioridades críticas.

    def end_turn(self):
        """Termina el turno, evaluando si debe ahorrar puntos."""
        if self.decide_to_save_points():
            self.saved_points = self.action_points
            if self.saved_points > 8:  # Limitar a un máximo de 8 puntos
                self.saved_points = 8

        else:
            self.saved_points = 0  # No ahorra puntos
        self.action_points = 0  # Reinicia los puntos para el siguiente turno

    def step(self):
        """Define el comportamiento del agente en cada paso."""
        self.calculate_action_points()

        # Priorizar acciones según el entorno
        while self.action_points > 0:
            # Aquí se puede incluir lógica de movimiento o interacción
            break
        # Finaliza el turno:
        self.end_turn()

In [ ]:
class Mansion(Model):
    def __init__(self, width, height, luigi, ghost, walls, doors, doors_connected, entrances, portrait):
        super().__init__()

In [ ]:
from flask import Flask, jsonify
from collections import OrderedDict

app = Flask(__name__)

def procesar_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Procesar paredes
    WALLS = []
    index = 0
    for _ in range(6):  # 6 líneas de paredes
        row = []
        cells = lines[index].strip().split()
        for cell in cells:
            # Cada celda es un diccionario con paredes explícitas
            walls = {
                "top": int(cell[0]),
                "left": int(cell[1]),
                "bottom": int(cell[2]),
                "right": int(cell[3])
            }
            row.append(walls)
        WALLS.append(row)
        index += 1

    # Procesar puntos de interés
    FAKE_ALARMS = []
    VICTIMS = []
    for _ in range(3):  # Máximo 3 líneas para puntos de interés
        r, c, marker_type = lines[index].strip().split()
        if marker_type == 'f':  # Falsa alarma
            FAKE_ALARMS.append({"row": int(r), "col": int(c)})
        elif marker_type == 'v':  # Víctima
            VICTIMS.append({"row": int(r), "col": int(c)})
        index += 1

    # Procesar marcadores de fuego
    GHOSTS = []
    for _ in range(10):  # 10 líneas de fuego
        r, c = map(int, lines[index].strip().split())
        GHOSTS.append({"row": r, "col": c})
        index += 1

    # Procesar marcadores de puertas
    DOORS = []
    for _ in range(8):  # 8 líneas de puertas
        r1, c1, r2, c2 = map(int, lines[index].strip().split())
        DOORS.append({"r1": r1, "c1": c1, "r2": r2, "c2": c2})
        index += 1

    # Procesar puntos de entrada
    ENTRANCES = []
    for _ in range(4):  # 4 líneas de puntos de entrada
        r, c = map(int, lines[index].strip().split())
        ENTRANCES.append({"row": r, "col": c})
        index += 1

    # Usar OrderedDict para asegurar el orden de las claves en el JSON
    data = OrderedDict([
        ("walls", WALLS),
        ("fake_alarms", FAKE_ALARMS),
        ("victims", VICTIMS),
        ("fires", GHOSTS),
        ("doors", DOORS),
        ("entrances", ENTRANCES),
        ("width", 8),
        ("height", 6)
    ])
    
    return data

@app.route('/get_board', methods=['GET'])
def get_board():
    file_path = './final.txt'  # Ruta del archivo
    data = procesar_txt(file_path)
    return jsonify(data)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)  # Ejecuta en el puerto 5000



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.155:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Nov/2024 21:17:07] "GET /get_board HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2024 22:28:49] "GET /get_board HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2024 22:30:27] "GET /get_board HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2024 22:31:24] "GET /get_board HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2024 22:32:00] "GET /get_board HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2024 22:33:36] "GET /get_board HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2024 22:44:53] "GET /get_board HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2024 22:49:41] "GET /get_board HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2024 22:52:21] "GET /get_board HTTP/1.1" 200 -
